In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout, Concatenate
from keras.layers import LSTM, Embedding, Bidirectional, GRU
from keras.layers import SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.initializers import Constant
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
import pickle
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import collections
from sklearn.utils.class_weight import compute_class_weight
from gensim import models

In [ ]:
data_train = pd.read_csv('/content/drive/My Drive/ML/Test Thesis/data_train.csv')
data_val = pd.read_csv('/content/drive/My Drive/ML/Test Thesis//data_val.csv')
data_test = pd.read_csv('/content/drive/My Drive/ML/Test Thesis/data_test.csv')

data_train.drop('Unnamed: 0', axis = 1, inplace = True)
data_val.drop('Unnamed: 0', axis = 1, inplace = True)
data_test.drop('Unnamed: 0', axis = 1, inplace = True)

PhoBERT

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModel.from_pretrained("vinai/phobert-base-v2")

def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs[0][:, 0, :].squeeze().numpy()
    return embedding

embeddings = data_train['processed_content'].apply(get_embedding)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
np.save('/content/drive/MyDrive/Test Thesis/embeddings_train_PhoBert.npy', embeddings)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs[0][:, 0, :].squeeze().numpy()
    return embedding

embeddings = data_val['processed_content'].apply(get_embedding)
np.save('/content/drive/MyDrive/Test Thesis/embeddings_val_PhoBert.npy', embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs[0][:, 0, :].squeeze().numpy()
    return embedding

embeddings = data_test['processed_content'].apply(get_embedding)
np.save('/content/drive/MyDrive/Test Thesis/embeddings_test_PhoBert.npy', embeddings)

SimCSE

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from pyvi.ViTokenizer import tokenize
from tqdm import tqdm
import numpy as np

df = {
    'train': data_train,
    'val': data_val,
    'test': data_test
}

PhobertTokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

def process_batch(model, tokenizer, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
    return outputs.cpu().numpy()
batch_size = 200
embeddings = {}

for state in tqdm(['train','val','test']):
    sentences = df[state].processed_content.values
    state_embeddings = []
    for i in tqdm(range(0, len(sentences), batch_size)):
        batch_sentences = sentences[i:i+batch_size]
        batch_sentences = [tokenize(sentence) for sentence in batch_sentences]
        batch_embeddings = process_batch(model, PhobertTokenizer, batch_sentences)
        state_embeddings.append(batch_embeddings)
    embeddings[state] = np.concatenate(state_embeddings, axis=0)

np.save('/content/drive/MyDrive/Thesis/embeddings_train.npy', embeddings['train'])
np.save('/content/drive/MyDrive/Thesis/embeddings_train.npy', embeddings['val'])
np.save('/content/drive/MyDrive/Thesis/embeddings_train.npy', embeddings['test'])